## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-31-22-52-40 +0000,nyt,Search Suspended for 6 Missing in Boat Off the...,https://www.nytimes.com/2026/01/30/us/fishing-...
1,2026-01-31-22-49-55 +0000,nypost,Mamdani to sign bill barring ICE from operatin...,https://nypost.com/2026/01/31/us-news/mamdani-...
2,2026-01-31-22-44-01 +0000,nypost,UK Prime Minister calls on ex-royal Andrew to ...,https://nypost.com/2026/01/31/world-news/uk-pr...
3,2026-01-31-22-37-14 +0000,latimes,Woman sentenced to state prison in fentanyl mu...,https://www.latimes.com/california/story/2026-...
4,2026-01-31-22-25-03 +0000,nypost,Venezuela set to release hundreds of political...,https://nypost.com/2026/01/31/world-news/venez...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2441/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
11,ice,28
81,trump,26
29,epstein,14
53,judge,11
75,new,10


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
9,2026-01-31-22-14-54 +0000,bbc,Judge orders release of five-year-old detained...,https://www.bbc.com/news/articles/cm2ygq5p2l3o...,69
6,2026-01-31-22-19-56 +0000,wapo,Judge orders release of 5-year-old detained by...,https://www.washingtonpost.com/nation/2026/01/...,67
35,2026-01-31-20-52-55 +0000,cbc,"Judge orders U.S. to free 5-year-old boy, fath...",https://www.cbc.ca/news/world/father-son-relea...,67
5,2026-01-31-22-23-20 +0000,nypost,ICE agents chase down migrant sex predator aft...,https://nypost.com/2026/01/31/us-news/ice-agen...,66
138,2026-01-31-10-30-00 +0000,wsj,Trump’s pick for Fed chair has long attacked t...,https://www.wsj.com/economy/central-banking/ke...,66


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
9,69,2026-01-31-22-14-54 +0000,bbc,Judge orders release of five-year-old detained...,https://www.bbc.com/news/articles/cm2ygq5p2l3o...
138,66,2026-01-31-10-30-00 +0000,wsj,Trump’s pick for Fed chair has long attacked t...,https://www.wsj.com/economy/central-banking/ke...
159,54,2026-01-31-05-07-00 +0000,wsj,The Senate passed a bipartisan deal Friday to ...,https://www.wsj.com/politics/policy/senate-set...
62,45,2026-01-31-18-50-44 +0000,nyt,"New Epstein Files Name Elon Musk, Bill Gates a...",https://www.nytimes.com/2026/01/31/us/epstein-...
201,38,2026-01-31-00-50-39 +0000,bbc,I mocked the Saudi leader on YouTube - then my...,https://www.bbc.com/news/articles/cj6w3zgden0o...
163,34,2026-01-31-04-03-13 +0000,eltecolote,"Students, teachers walk out as San Francisco j...",https://eltecolote.org/content/en/ice-walkout-...
34,32,2026-01-31-20-57-34 +0000,nypost,Israel reportedly kills 32 in airstrikes in Ga...,https://nypost.com/2026/01/31/world-news/israe...
59,31,2026-01-31-19-21-58 +0000,nypost,"TV star fisherman, crew missing off Massachuse...",https://nypost.com/2026/01/31/us-news/tv-star-...
17,29,2026-01-31-21-57-06 +0000,nypost,Stanley Richards named to lead Department of C...,https://nypost.com/2026/01/31/us-news/stanley-...
191,28,2026-01-31-02-00-00 +0000,wsj,Shifting views on the president’s immigration ...,https://www.wsj.com/politics/elections/the-tru...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
